# Часть 3. Who's the mightiest of them all?
## Постройте 4 варианта градиентного бустинга, используя значения гиперпараметров “из коробки”: реализация из sklearn, XGBoost, CatBoost, LightGBM

In [9]:
# ! pip install lightgbm
# ! pip install catboost
# pip install xgboost

import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier 
from sklearn.metrics import roc_auc_score, f1_score

df_train = pd.read_csv(r'.\2.Split dataset to train-test\train.csv')
df_test = pd.read_csv(r'.\2.Split dataset to train-test\test.csv')

X_train, y_train = df_train.drop('Churn', axis=1), df_train['Churn']
X_test, y_test = df_test.drop('Churn', axis=1), df_test['Churn']

#print(X_train.head())
#print(y_train.head())
#print(X_test.head())
#print(y_test.head())

def simple_test(model):
    print(model)
    model.fit(X_train, y_train)
    #y_pred = model.predict(X_train)
    #print('F1 score на тренировочной выборке\t: {:.3f}'.format(f1_score(y_train, y_pred)))
    #print("AUC-ROC на тренировочной выборке\t: {:.3f}".format(roc_auc_score(y_train, y_pred)))
    y_pred = model.predict(X_test)
    print('F1 score на тестовой выборке\t: {:.3f}'.format(f1_score(y_test, y_pred)))
    print("AUC-ROC на тестовой выборке\t: {:.3f}".format(roc_auc_score(y_test, y_pred)))


model = GradientBoostingClassifier(random_state=1)
simple_test(model)
#print(model.get_params())
print()

model = XGBClassifier(random_state=1)
simple_test(model)
#print(model.get_xgb_params())
print()

model =  CatBoostClassifier(random_state=1, logging_level="Silent")
simple_test(model)
#print(model.get_all_params())
print()

model = LGBMClassifier(random_state=1, verbose=-1)
simple_test(model)
#print(model.get_params())



GradientBoostingClassifier(random_state=1)
F1 score на тестовой выборке	: 0.563
AUC-ROC на тестовой выборке	: 0.700

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=1, ...)
F1 score на тестовой выборке	: 0.540
AUC-ROC на тестовой выборке	: 0.687

F1 score на тестовой выборке	: 0.578
AUC-ROC на тестов

### Кто лидирует

Лучше всего метрики f1, AUC-ROC у CatBoostClassifier.

## Настройка гиперпараметров моделей на кросс-валидации
Подберем набор гиперпараметров по кросс-валидационной оценке по тренировочной выборке.
После этого прогоним финишный тест на отложенной тестовой выборке.

In [24]:
# ! pip install lightgbm
# ! pip install catboost
# pip install xgboost

import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier 
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import GridSearchCV

df_train = pd.read_csv(r'.\2.Split dataset to train-test\train.csv')
df_test = pd.read_csv(r'.\2.Split dataset to train-test\test.csv')

X_train, y_train = df_train.drop('Churn', axis=1), df_train['Churn']
X_test, y_test = df_test.drop('Churn', axis=1), df_test['Churn']

scoring = 'f1'

#print(X_train.head())
#print(y_train.head())
#print(X_test.head())
#print(y_test.head())

def print_finish_metrics(model, clf):
    print(model)
    y_pred = clf.predict(X_test)
    print('F1 score на финишной тестовой выборке \t: {:.3f}'.format(f1_score(y_test, y_pred)))
    print("AUC-ROC на финишной тестовой выборке \t: {:.3f}".format(roc_auc_score(y_test, y_pred)))
    print()

def RunGridSearchCV(model, parameters, is_finish = False):
    print('Подбор гиперпараметров:', model)
    clf = GridSearchCV(model, parameters, scoring = scoring, cv = 5, refit=True, n_jobs=-1)
    clf.fit(X_train, y_train)
    print(scoring, ' на кросс-валидации \t: {:.3f}'.format(clf.best_score_))
    print('Best parameters: ', clf.best_params_)
    return clf
    

parameters = {'learning_rate': [0.1, 0.01, 0.001],  
              'n_estimators':[10, 100, 200],
              'max_depth': [1, 2],
              'max_features':[None, 'sqrt'],
              'subsample': [0.5, 1.0],
              'loss': ['log_loss', 'exponential']
              }
model = GradientBoostingClassifier(random_state=1)
clf_GradientBoostingClassifier = RunGridSearchCV(model, parameters)

Подбор гиперпараметров: GradientBoostingClassifier(random_state=1)
f1  на кросс-валидации 	: 0.590
Best parameters:  {'learning_rate': 0.1, 'loss': 'exponential', 'max_depth': 2, 'max_features': 'sqrt', 'n_estimators': 100, 'subsample': 0.5}


In [27]:
parameters = {'learning_rate': [0.1, 0.01, 0.001],  
              'n_estimators':[10, 100, 200],
              'max_depth':[1, 2],
              'subsample': [0.5, 1.0],
              'objective': ['binary:logistic', 'binary:hinge']
              }
model = XGBClassifier(random_state=1)
clf_XGBClassifier = RunGridSearchCV(model, parameters)

Подбор гиперпараметров: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=1, ...)
f1  на кросс-валидации 	: 0.588
Best parameters:  {'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 200, 'objective': 'binary:logistic', 'subsample': 0.5}


In [28]:
parameters = {'learning_rate': [0.1, 0.01, 0.001],  
              'iterations':[10, 50, 100],
              'subsample': [0.5, 1.0],
              'depth': [1, 2, 4],
              'loss_function': ['Logloss', 'CrossEntropy']
              }
model =  CatBoostClassifier(random_state=1, logging_level="Silent", thread_count=8)
clf_CatBoostClassifier = RunGridSearchCV(model, parameters)

Подбор гиперпараметров: <catboost.core.CatBoostClassifier object at 0x00000286720EFA50>
f1  на кросс-валидации 	: 0.589
Best parameters:  {'depth': 4, 'iterations': 100, 'learning_rate': 0.1, 'loss_function': 'Logloss', 'subsample': 1.0}


In [34]:
parameters = {'learning_rate': [0.1, 0.075, 0.01, 0.001], 
              'n_estimators':[5, 10, 25, 30, 40],
              'subsample': [0.5, 0.75, 1.0],
              'max_depth': [1, 2, 3],
              'objective': ['binary', 'cross_entropy', None]
              }
model = LGBMClassifier(random_state=1, metric='f1', verbose=-1, n_jobs=-1)
clf_LGBMClassifier = RunGridSearchCV(model, parameters)

Подбор гиперпараметров: LGBMClassifier(metric='f1', n_jobs=-1, random_state=1, verbose=-1)
f1  на кросс-валидации 	: 0.576
Best parameters:  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 40, 'objective': 'binary', 'subsample': 0.5}


### Финишный тест моделей
Прогоним тест на финишной отложенной тестовой выборке

In [35]:
print_finish_metrics("GradientBoostingClassifier", clf_GradientBoostingClassifier)
print_finish_metrics("XGBClassifier", clf_XGBClassifier)
print_finish_metrics("CatBoostClassifier", clf_CatBoostClassifier)
print_finish_metrics("LGBMClassifier", clf_LGBMClassifier)

GradientBoostingClassifier
F1 score на финишной тестовой выборке 	: 0.562
AUC-ROC на финишной тестовой выборке 	: 0.700

XGBClassifier
F1 score на финишной тестовой выборке 	: 0.561
AUC-ROC на финишной тестовой выборке 	: 0.699

CatBoostClassifier
F1 score на финишной тестовой выборке 	: 0.568
AUC-ROC на финишной тестовой выборке 	: 0.703

LGBMClassifier
F1 score на финишной тестовой выборке 	: 0.544
AUC-ROC на финишной тестовой выборке 	: 0.688



### Кто лидирует

Лучше всего метрики f1, AUC-ROC у CatBoostClassifier.
Но абсолютный победитель: CatBoostClassifier "из коробки".

## Эксперимент: логистическая регрессия
На этапе EDA был применен Метод Внедрения с логистической регрессией. Наблюдение показало, что логистическая регрессия хорошо обучилась по набору данных.
Поэтому проверим результаты с логистической регрессией.

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score

df_train = pd.read_csv(r'.\2.Split dataset to train-test\train.csv')
df_test = pd.read_csv(r'.\2.Split dataset to train-test\test.csv')

X_train, y_train = df_train.drop('Churn', axis=1), df_train['Churn']
X_test, y_test = df_test.drop('Churn', axis=1), df_test['Churn']

model_log_lr1 = LogisticRegression(C=1, solver='liblinear', penalty='l2', random_state=1)
model_log_lr1.fit(X_train, y_train)

y_pred = model_log_lr1.predict(X_test)

print('F1 score на тестовой выборке\t: {:.3f}'.format(f1_score(y_test, y_pred)))
print("AUC-ROC на тестовой выборке: {:.3f}".format(roc_auc_score(y_test, y_pred)))


F1 score на тестовой выборке	: 0.569
AUC-ROC на тестовой выборке: 0.704


*Вывод по эксперименту: логистическая регрессия на данном наборе данных показала себя на уровне "бустингов".*

## Эксперимент 2: Bagging из "бустингов"
"Бустинги" дают низкий Bias, а Bagging уменьшает Variance. Могут ли они дать лучше результаты вместе: низкий Bias и Variance.

In [37]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier 
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import roc_auc_score, f1_score

df_train = pd.read_csv(r'.\2.Split dataset to train-test\train.csv')
df_test = pd.read_csv(r'.\2.Split dataset to train-test\test.csv')

X_train, y_train = df_train.drop('Churn', axis=1), df_train['Churn']
X_test, y_test = df_test.drop('Churn', axis=1), df_test['Churn']

model = BaggingClassifier(estimator=LGBMClassifier(random_state=1, max_depth=7, n_estimators=100, verbose=-1), 
                          n_jobs=-1, 
                          n_estimators=100,
                          random_state=1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print('F1 score на тестовой выборке\t: {:.3f}'.format(f1_score(y_test, y_pred)))
print("AUC-ROC на тестовой выборке\t: {:.3f}".format(roc_auc_score(y_test, y_pred)))

F1 score на тестовой выборке	: 0.570
AUC-ROC на тестовой выборке	: 0.704


*Вывод по эксперименту: качество не поднялось.*